<a href="https://colab.research.google.com/github/1028Luo/LLM-Domain-Specific-Assistant/blob/main/finetune_LLAMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning LLAMA

In [1]:
# install
!pip install -q accelerate==1.2.0
!pip install -q peft==0.14.0
!pip install -q bitsandbytes==0.45.0
!pip install -q transformers==4.47.1
!pip install -q trl==0.13.0
!pip install -q huggingface_hub
!pip install -q datasets==3.2.0
!pip install numba


# import
import os
import torch

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig
from huggingface_hub import login
from numba import cuda

device = 'cuda'  # Use GPU

# Log in to Hugging Face
from google.colab import userdata
my_hugging_face_token = userdata.get('huggingface_token')
login(token=my_hugging_face_token)





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
# load model and tokenizer
model_name = "meta-llama/Llama-3.2-3B"
#model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
#tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

new_model = "/llama-3-3b-workout"

# dataset
dataset_name = "Jiexing1028/workout-plan"
dataset = load_dataset(dataset_name, split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/309 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/155k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/147 [00:00<?, ? examples/s]

In [3]:
# examine dataset and format it

print(dataset)

LLAMA_31_prompt_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{sys_msg}<|eot_id|><|start_header_id|>user<|end_header_id|>

{user_msg}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{asst_msg}<|eot_id|>
"""


def formatting_prompts_func(example): # change this to fit data format of other model
    output_texts = []
    for i in range(len(example['question'])):
        #text = f"### Question: {example['question'][i]}\n ### Answer: {example['answer'][i]}"
        text = LLAMA_31_prompt_template.format(sys_msg="you are a helpful assistant in generating personalised workout plans", user_msg= example['question'][i], asst_msg= {example['answer'][i]})
        output_texts.append(text)
    return output_texts


temp = formatting_prompts_func(dataset)
print(temp)

Dataset({
    features: ['question', 'answer'],
    num_rows: 147
})
["\n<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nyou are a helpful assistant in generating personalised workout plans<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI'm 35 years old and weigh 150 pounds, what kind of workout plan is best for my age group to lose weight?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{'**Personalized Workout Plan for a 35-year-old weighing 150 pounds**\\n\\nAs we age, our metabolism slows down, and losing weight can become more challenging. Given your age and weight, I recommend a workout plan that combines cardiovascular exercises with strength training to boost your metabolism and burn fat.\\n\\n**Recommended Workout Plan:**\\n\\n1. **Warm-up (5-10 minutes)**: Light cardio such as jogging, cycling, or rowing to get your heart rate up and loosen your muscles.\\n2. **Resistance Training (30-40 minutes, 3 times a week)**:\\n\\t* Squats (3 sets of 12 re

In [4]:
# Parameters

######## Lora ########
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

######## Quantization using bitsandbytes ########

# Qlora stores model in 4 bit
use_4bit = True

# Quantization type (fp4 or nf4)
# fp4 is float-point 4, nf4 is a variant of fp4 tailored for deep learning
bnb_4bit_quant_type = "nf4"

# Dequantizes weights from 4 bit to 16 bit when they are needed for computation,
# hence the low vram usage
# Use higher precision in computation for numerical stability
bnb_4bit_compute_dtype = "float16"

# Double quantization
use_nested_quant = False

# compute type becomes torch.float16
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

######## SFTTrainer ########
_SFTConfig = SFTConfig(
    output_dir=new_model,
    dataset_text_field="text", # will be used only when no formatting function
    max_seq_length=256,
    report_to="tensorboard", # by default needs login to wandb
    num_train_epochs = 5,
    logging_steps = 25, # log traning loss
    gradient_checkpointing = True, # reduce VRAM use by only saving some gradient
    warmup_ratio = 0.03, # gradually increase lr in the first 3% of steps, helps avoid training instability
    lr_scheduler_type = "cosine", # lr starts at max and then drop to near 0 following a cosine path
    weight_decay = 0.001, # a term in the loss function that descourage high value of weights
    optim = "paged_adamw_32bit" # a variant of adam using page to better manage memory
    )


In [5]:





# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training



trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    args=_SFTConfig,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
)

trainer.train()

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Map:   0%|          | 0/147 [00:00<?, ? examples/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,2.193100
50,1.993500
75,1.872900


TrainOutput(global_step=95, training_loss=1.9798920079281455, metrics={'train_runtime': 1078.6447, 'train_samples_per_second': 0.681, 'train_steps_per_second': 0.088, 'total_flos': 3187432182251520.0, 'train_loss': 1.9798920079281455, 'epoch': 5.0})

In [22]:
LLAMA_31_chat_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{sys_msg}<|eot_id|><|start_header_id|>user<|end_header_id|>

{user_msg}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# AAA: need prompt template here? tokenize prompt here to see what it looks like
# test inference
prompt = "I'm 25 years old and have severe elbow injury, what exercises should I avoid to prevent further injury?"




baseline_model = "meta-llama/Llama-3.2-3B-instruct"
baseline_tokenizer = AutoTokenizer.from_pretrained(baseline_model)
baseline_model = AutoModelForCausalLM.from_pretrained(
    baseline_model,
    load_in_4bit=True)

pipe = pipeline(task="text-generation", model=baseline_model, tokenizer=baseline_tokenizer, max_length=200, return_full_text=True)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<s>[INST] I'm 25 years old and have severe elbow injury, what exercises should I avoid to prevent further injury? [/INST] 

The following exercises should be avoided:

*   **Weightlifting**: Avoid weightlifting, especially with heavy weights, as it can put excessive stress on the elbow joint.
*   **Throwing or impact activities**: Avoid throwing or impact activities, such as tennis, basketball, or football, which can exacerbate the injury.
*   **Overhead activities**: Avoid overhead activities, such as lifting heavy objects or using tools, which can put strain on the elbow joint.
*   **Repetitive motion**: Avoid repetitive motion activities, such as typing or using a mouse, which can cause strain on the elbow joint.

The following exercises may be modified or avoided:

*   **Elbow flexion exercises**: Avoid exercises that involve bending the elbow, such as arm circles or elbow raises.
*   **Shoulder flexion exercises**: Avoid exercises that


In [24]:

pipe = pipeline(task="text-generation", model=model, tokenizer=baseline_tokenizer, max_length=300, return_full_text=False)
#result = pipe(LLAMA_31_chat_template.format(sys_msg="you are a helpful assistant in generating personalised workout plans", user_msg=prompt))
result = pipe(prompt)
print(result[0]['generated_text'])





Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 How can I strengthen my elbow?
I'm 25 years old and have severe elbow injury, what exercises should I avoid to prevent further injury? How can I strengthen my elbow?
I'm 25 years old and have severe elbow injury, what exercises should I avoid to prevent further injury? How can I strengthen my elbow?
The elbow is a very complex joint and can be easily injured. The most common elbow injury is a sprain or strain of the ligaments. The most common elbow injury is a sprain or strain of the ligaments. The most common elbow injury is a sprain or strain of the ligaments. The most common elbow injury is a sprain or strain of the ligaments. The most common elbow injury is a sprain or strain of the ligaments. The most common elbow injury is a sprain or strain of the ligaments. The most common elbow injury is a sprain or strain of the ligaments. The most common elbow injury is a sprain or strain of the ligaments. The most common elbow injury is a sprain or strain of the ligaments. The most common 

In [ ]:
# 3B works much better than 1B
# TODO:
# 1. print pretrain's tokenizer.(promt) and instruct's, see what's the difference
# 2. find why the fine-tuned repeat itself


In [11]:
# Save trained model
trainer.push_to_hub()
#trainer.model.save_pretrained(push_to_hub=True, repo_name=new_model)
print('model pushed')

adapter_model.safetensors:   0%|          | 0.00/18.4M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1738734609.7a90c3045329.1247.0:   0%|          | 0.00/6.84k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

model pushed


In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

2464

In [12]:
# free VRAM
device = cuda.get_current_device()
device.reset()
print('finished')

finished
